In [1]:
pip install bertopic sentence-transformers umap-learn hdbscan

Python(17292) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer


Couldn't import dot_parser, loading of dot files will not be possible.


In [8]:
# Load file hasil preprocessing
df = pd.read_csv("PreprocessedReviews.csv")

# Validasi kolom
if 'Cleaned_Review' not in df.columns:
    raise ValueError("Kolom 'Cleaned_Review' tidak ditemukan.")

docs = df['Cleaned_Review'].astype(str).tolist()
print(f"Jumlah dokumen: {len(docs)}")


Jumlah dokumen: 9564


In [9]:
seed_topic_list = [
    ["mechanics","combat","controls","interaction"],
    ["bugs","lag","fps","crash","stability"],
    ["score","leaderboard","points","ranks","levels"],
    ["multiplayer","co","op","community","online"],
    ["character","npc","customization","design"],
    ["soundtrack","bgm","audio","music","melody"],
    ["plot","narrative","lore","ending","storyline"]
]


In [10]:
# Buat vectorizer dengan batasan kata minimum agar tidak terlalu umum
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

# Buat model BERTopic dengan seed topics
topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    language="english",
    seed_topic_list=seed_topic_list,
    calculate_probabilities=True,
    verbose=True
)


In [11]:
topics, probs = topic_model.fit_transform(docs)


2025-05-29 17:11:40,579 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/299 [00:00<?, ?it/s]

2025-05-29 17:12:19,105 - BERTopic - Embedding - Completed ✓
2025-05-29 17:12:19,106 - BERTopic - Guided - Find embeddings highly related to seeded topics.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-29 17:12:19,647 - BERTopic - Guided - Completed ✓
2025-05-29 17:12:19,648 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2025-05-29 17:12:34,400 - BERTopic - Dimensionality - Completed ✓
2025-05-29 17:12:34,401 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-29 17:12:37,781 - BERTopic - Cluster - Completed ✓
2025-05-29 17:12:37,790 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-05-29 17:12:38,642 - BERTopic - Representation - Completed ✓


In [12]:
# Lihat topik paling umum
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4664,-1_game_like_really_fun,"[game, like, really, fun, good, play, time, fe...",[decent game series nothing played game one an...
1,0,398,0_game_character_like_time,"[game, character, like, time, make, thing, ene...",[hades developer supergiant game graphic forge...
2,1,351,1_story_character_dialogue_game,"[story, character, dialogue, game, great, real...",[game old school adventure game visual novel u...
3,2,247,2_card_deck_card game_slay spire,"[card, deck, card game, slay spire, slay, spir...",[edit february check duelyst card castle inste...
4,3,217,3_downloadable_downloadable content_content_new,"[downloadable, downloadable content, content, ...",[downloadable content play daud assassinating ...
5,4,194,4_puzzle_puzzle game_bit_game,"[puzzle, puzzle game, bit, game, plot, short, ...",[nice design good puzzle however know primaril...
6,5,180,5_buy_sale_bought_worth,"[buy, sale, bought, worth, price, pay, money, ...",[recent price hike game received brazil severa...
7,6,138,6_music_soundtrack_player choice_sound,"[music, soundtrack, player choice, sound, audi...",[interesting game feel take seriously although...
8,7,105,7_building_city_builder_base,"[building, city, builder, base, city builder, ...",[good simple base building game low replay val...
9,8,98,8_horror_horror game_scary_scare,"[horror, horror game, scary, scare, horror exp...","[great survival horror game, good horror game ..."


In [13]:
# Tampilkan semua topik dan kata kuncinya
for i in range(len(topic_model.get_topics())):
    print(f"Topik {i}: {topic_model.get_topic(i)}")


Topik 0: [('game', 0.0055527381854693905), ('character', 0.005413083096817474), ('like', 0.005076540862637702), ('time', 0.004821561903386622), ('make', 0.004340007910324743), ('thing', 0.0043161289689396636), ('enemy', 0.004180318340952027), ('story', 0.0038362183861678557), ('feel', 0.0038321379491806757), ('combat', 0.0037856811597226214)]
Topik 1: [('story', 0.018558750448843624), ('character', 0.011099522047620575), ('dialogue', 0.009348844615175161), ('game', 0.008844231411703487), ('great', 0.007859425222961715), ('really', 0.007444358379604183), ('adventure', 0.007286540838511126), ('good', 0.0071791145309621565), ('like', 0.006676520937086403), ('ending', 0.006495344539417933)]
Topik 2: [('card', 0.042678709183227345), ('deck', 0.023656607933928094), ('card game', 0.010327230295038618), ('slay spire', 0.008202792396259066), ('slay', 0.008174813086045125), ('spire', 0.00814726576651147), ('deckbuilder', 0.007864103703019187), ('mechanic', 0.0070206453841812205), ('play', 0.0064

In [14]:
topic_model.visualize_topics()

In [15]:
df['Assigned_Topic'] = topics
df.to_csv("TopicAssignedReviews.csv", index=False)
print("File dengan topik tersimpan sebagai TopicAssignedReviews.csv")


File dengan topik tersimpan sebagai TopicAssignedReviews.csv


In [16]:
df.groupby('Assigned_Topic')['Review'].apply(lambda x: x.sample(2)).reset_index(drop=True)

0                                bring me back Overwatch
1      At this point I barely get any of the game's m...
2      A lot of times I feel 'meh' about a game and w...
3      Dear god, talk about First World Problems the ...
4      Nice story and nice settings. A really true ma...
                             ...                        
229    Still has a lot to develop, wait until properl...
230    I cannot stress how good enough is to be able ...
231    Edit: I do feel obligated to ask you guys to v...
232    Very simple gameplay that is surprisingly enjo...
233    Such a great match 3!  Simple mechanics with s...
Name: Review, Length: 234, dtype: object

In [22]:
pip install gensim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install gensim nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import necessary libraries
import pandas as pd
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer

# Load the dataset
df = pd.read_csv("PreprocessedReviews.csv")

# Validate columns
if 'Cleaned_Review' not in df.columns:
    raise ValueError("Column 'Cleaned_Review' not found.")

# Extract documents for modeling
docs = df['Cleaned_Review'].astype(str).tolist()

# Define seed topics
seed_topic_list = [
    ["mechanics", "combat", "controls", "interaction"],
    ["bugs", "lag", "fps", "crash", "stability"],
    ["score", "leaderboard", "points", "ranks", "levels"],
    ["multiplayer", "co", "op", "community", "online"],
    ["character", "npc", "customization", "design"],
    ["soundtrack", "bgm", "audio", "music", "melody"],
    ["plot", "narrative", "lore", "ending", "storyline"]
]

# Create a vectorizer
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

# Fit the BERTopic model
topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    language="english",
    seed_topic_list=seed_topic_list,
    calculate_probabilities=True,
    verbose=True
)

topics, _ = topic_model.fit_transform(docs)

# Preprocess Documents for Coherence Evaluation
documents_per_topic = pd.DataFrame({"Topic": topics, "Document": docs})
documents_per_topic = documents_per_topic.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic['Document'].values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
            for topic in range(len(set(topics))-1)]

# Evaluate Coherence
coherence_model = CoherenceModel(topics=topic_words, 
                              texts=tokens, 
                              corpus=corpus,
                              dictionary=dictionary, 
                              coherence='c_v')
coherence = coherence_model.get_coherence()

print("Coherence Score:", coherence)


Couldn't import dot_parser, loading of dot files will not be possible.


2025-05-29 21:51:52,928 - BERTopic - Embedding - Transforming documents to embeddings.
Python(40683) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Batches:   0%|          | 0/299 [00:00<?, ?it/s]

2025-05-29 21:52:35,081 - BERTopic - Embedding - Completed ✓
2025-05-29 21:52:35,084 - BERTopic - Guided - Find embeddings highly related to seeded topics.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-29 21:52:35,642 - BERTopic - Guided - Completed ✓
2025-05-29 21:52:35,643 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2025-05-29 21:52:49,921 - BERTopic - Dimensionality - Completed ✓
2025-05-29 21:52:49,923 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-29 21:52:54,223 - BERTopic - Cluster - Completed ✓
2025-05-29 21:52:54,247 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-05-29 21:52:55,150 - BERTopic - Representation - Completed ✓
Python(40786) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explici

Coherence Score: 0.4845994549696664


In [2]:
topic_model.visualize_topics()

In [50]:
# Visualize documents in 3D
topic_model.visualize_documents(docs)


In [51]:
topic_model.visualize_barchart()

In [52]:
topic_model.visualize_heatmap()

# Experiment

In [ ]:
# Import necessary libraries
import pandas as pd
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from sentence_transformers import SentenceTransformer

# Load the dataset
df = pd.read_csv("PreprocessedReviews.csv")

# Validate columns
if 'Cleaned_Review' not in df.columns:
    raise ValueError("Column 'Cleaned_Review' not found.")

# Extract documents for modeling
docs = df['Cleaned_Review'].astype(str).tolist()

# Define seed topics
seed_topic_list = [
    ["mechanics", "combat", "controls", "interaction"],
    ["bugs", "lag", "fps", "crash", "stability"],
    ["score", "leaderboard", "points", "ranks", "levels"],
    ["multiplayer", "co", "op", "community", "online"],
    ["character", "npc", "customization", "design"],
    ["soundtrack", "bgm", "audio", "music", "melody"],
    ["plot", "narrative", "lore", "ending", "storyline"]
]

# Create a vectorizer
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

# Set UMAP model for 3D projection
umap_model = UMAP(n_neighbors=15, n_components=3, min_dist=0.1, metric='cosine')

# Choose SentenceTransformer model for embedding
embedding_model = SentenceTransformer("paraphrase-MPNet-base-v2")

# Fit the BERTopic model with custom UMAP and embedding model
topic_model = BERTopic(
    umap_model=umap_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    language="english",
    seed_topic_list=seed_topic_list,
    calculate_probabilities=True,
    verbose=True
)

# Fit the model and get topics
topics, _ = topic_model.fit_transform(docs)

# Check the first few topics to ensure it's iterable and correct
print(topics[:10])  # Display the first 10 topics

# Preprocess Documents for Coherence Evaluation
documents_per_topic = pd.DataFrame({"Topic": topics, "Document": docs})
documents_per_topic = documents_per_topic.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic['Document'].values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
            for topic in range(len(set(topics))-1)]

# Evaluate Coherence
coherence_model = CoherenceModel(topics=topic_words, 
                                  texts=tokens, 
                                  corpus=corpus,
                                  dictionary=dictionary, 
                                  coherence='c_v')
coherence = coherence_model.get_coherence()

print("Coherence Score:", coherence)


# Visualize topics with intertopic distance map
topic_model.visualize_topics()


2025-05-29 21:24:18,253 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/299 [00:00<?, ?it/s]

In [58]:
topic_model.visualize_topics()